In [20]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import numpy as np

from utils.metrics import ndcg_scorer

In [17]:
path = '../data/processed/'
train_users = pd.read_csv(path + 'processed_train_users.csv').head(50000)

In [18]:
y_train = train_users['country_destination']
train_users.drop('country_destination', axis=1, inplace=True)
train_users.drop('id', axis=1, inplace=True)
train_users = train_users.fillna(-1)
x_train = train_users.values
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [19]:
xgb = XGBClassifier(
    max_depth=1,
    learning_rate=1,
    n_estimators=1,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=1,
    colsample_bytree=1,
    colsample_bylevel=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    missing=None,
    silent=True,
    nthread=-1,
    seed=42
)

kf = KFold(len(x_train), n_folds=10, random_state=42)

score = cross_val_score(xgb, x_train, encoded_y_train, cv=kf, scoring=ndcg_scorer)

In [39]:
model = SelectFromModel(ExtraTreesClassifier())
model.fit(train_users, encoded_y_train)
X_new = model.transform(train_users)

In [45]:
model.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [41]:
print x_train.shape
print X_new.shape

(50000, 673)
(50000, 45)


In [34]:
score = cross_val_score(xgb, X_new, encoded_y_train, cv=kf, scoring=ndcg_scorer)

In [35]:
np.mean(score)

0.77182916021073189

In [23]:
np.mean(score)

0.77182426784182623